### Final Project: N-Body Simulation of Gravitational Collapse
#### ThomasJae Garcia

In [ ]:
import math
import numpy as np
import numpy.random as rand
import matplotlib
import matplotlib.pyplot as plt
import scipy.constants as const
import numpy.fft as fft

#### Initial Conditions

In [ ]:
Np = 4096
N = 1024
L = 1
R = L/4
m = .1
G = const.G

rho_average = (Np * m) / (math.pi * R**2)

t_dyn = math.sqrt((math.pi * R) / (8 * G * rho_average))

#### Create Initial Positions of Points

In [ ]:
xpoints = []
ypoints = []
points = []
start = L/2

#Create set of evenly distributed points on a circle
rand.seed(728)
for i in range(Np):
        r = R*math.sqrt(rand.random())
        angle = 2*math.pi*rand.random()
        x = start + r*math.cos(angle)
        y = start + r*math.sin(angle)
        points.append((x,y))
        xpoints.append(x)
        ypoints.append(y)

#Scatter points
plt.figure(figsize=(5,5)) #make all figures larger
plt.scatter(xpoints, ypoints, s=1)
plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title("Initial Points")
plt.axes().set_aspect('equal')
plt.show()

#### Create Density Field (CIC)

In [ ]:
cell_x = L/1024        
cell_y = L/1024

def density_field(points):
    
    m = .1
    mass = np.zeros((1024, 1024))
    A = cell_x * cell_y
    center_cell = L/2048
    
    for i in range(Np):
        
        #Find where particle is in the the cell
        cx, cy = points[i][0] % cell_x , points[i][1] % cell_y
        
        #Find which cell the particle is in
        loc_y, loc_x = int(points[i][0] // cell_x) , int(points[i][1] // cell_y)
    
        #Add mass to cell the particle is in
        center_diff_x = abs(center_cell - cx)
        center_diff_y = abs(center_cell - cy)
        mass[loc_x][loc_y] += (cell_x-center_diff_x)*(cell_y-center_diff_y) * m/A
        
        #Add mass to surrounding particles
        
        #Upper right
        if cx > cell_x and cy > cell_y:
            mass[(loc_x + 1) % N][loc_y] += (cell_x - center_diff_x )*(center_diff_y) *  m/A
            mass[loc_x][(loc_y + 1) % N] += (center_diff_x)*(cell_y - center_diff_y) * m/A
            mass[(loc_x + 1) % N][(loc_y + 1) % N] += (center_diff_x)*(center_diff_y) * m/A
        
        #Lower right
        elif cx > cell_x and cy < cell_y:
            mass[loc_x - 1][loc_y] += (cell_x - center_diff_x )*(center_diff_y) * m/A
            mass[loc_x][(loc_y + 1) % N] += (center_diff_x)*(cell_y - center_diff_y) * m/A
            mass[loc_x - 1][(loc_y + 1) % N] += (center_diff_x)*(center_diff_y) * m/A
            
        #Upper left    
        elif cx < cell_x and cy > cell_y:
            mass[(loc_x + 1) % N][loc_y] += (cell_x - center_diff_x )*(center_diff_y) * m/A
            mass[loc_x][loc_y - 1] += (center_diff_x)*(cell_y - center_diff_y) * m/A
            mass[(loc_x + 1) % N][loc_y - 1] += (center_diff_x)*(center_diff_y) * m/A
        
        #Lower left
        else:
            mass[loc_x - 1][loc_y] += (cell_x - center_diff_x )*(center_diff_y) * m/A
            mass[loc_x][loc_y - 1] += (center_diff_x)*(cell_y - center_diff_y) * m/A
            mass[loc_x - 1][loc_y - 1] += (center_diff_x)*(center_diff_y) * m/A
            
    rho = mass / A
            
    return rho

rho = density_field(points)

plt.figure(figsize=(5,5))
plt.title("Density Field")
plt.imshow(rho, 'hot_r')
plt.show()

#### Construct Kernel

In [ ]:
def get_w(points):
    
    r = np.zeros((1024, 1024))
    w = np.zeros((1024, 1024))
    x = np.zeros(1024)
    y = np.zeros(1024)
    
    for i in range(len(r)):
        
        if i < 1024//2:
            y[i] = L/1024 * i
        else:
            y[i] = L/1024 * (i - 1024)
            
        for j in range(len(r)):
            
            if j < 1024//2:
                x[j] = L/1024 * j
                r[i,j] = math.sqrt(x[j]**2+y[i]**2)
            else:
                x[j] = L/1024 * (j - 1024)
                r[i,j] = math.sqrt(x[j]**2+y[i]**2)
                
            if i == 0 and j == 0:
                w[0,0] = -G/.4
            else:
                w[i,j] = -G / r[i,j]
    
    return w

kernel = fft.rfft2(get_w(points))

 #### Calculate Gravitational Potential

In [ ]:
rho_k = fft.rfft2(rho)

phi_k = rho_k * kernel * (L/N)**2 

phi = fft.irfft2(phi_k)

plt.figure(figsize=(5,5))
plt.title("Gravitational Potential Field")
plt.imshow(phi)
plt.show()

#### Interpolate Forces

In [ ]:
def grad_phi(phi, i, j, h):
    #Deal with boundaries using modulus
    (i, i_p, i_m, j, j_p, j_m) = (i % N, (i+1) % N, (i-1) % N, j % N, (j+1) % N, (j-1) % N)
    #Central difference the field
    return -np.array([(phi[i][j_p] - phi[i][j_m])/(2*h), (phi[i_p][j] - phi[i_m][j])/(2*h)])

def particle_forces(phi, points):
    
    m = .1
    force = np.zeros((4096, 2), dtype=float)
    h = L/1024
    A = cell_x * cell_y
    center_cell = L/2048
    
    for i in range(Np):
        
        #Find where particle is in the the cell
        cx, cy = points[i][0] % cell_x , points[i][1] % cell_y
        
        #Find which cell the particle is in
        loc_y, loc_x = int(points[i][0] // cell_x) , int(points[i][1] // cell_y)
    
        #Add force from parent cell
        center_diff_x = abs(center_cell - cx)
        center_diff_y = abs(center_cell - cy)
        force[i] += (cell_x-center_diff_x)*(cell_y-center_diff_y) * grad_phi(phi, loc_x, loc_y, h)/A
        
        #Add force from surrounding cells
        
        #Upper right
        if cx > cell_x and cy > cell_y:
            force[i] += ((cell_x - center_diff_x )*(center_diff_y) * grad_phi(phi, loc_x + 1, loc_y, h)/A
                       + (center_diff_x)*(cell_y - center_diff_y) * grad_phi(phi, loc_x, loc_y + 1, h)/A
                       + (center_diff_x)*(center_diff_y) * grad_phi(phi, loc_x + 1, loc_y + 1, h)/A)
        
        #Lower right
        elif cx > cell_x and cy < cell_y:
            force[i] += ((cell_x - center_diff_x )*(center_diff_y) * grad_phi(phi, loc_x - 1, loc_y, h)/A
                       + (center_diff_x)*(cell_y - center_diff_y) * grad_phi(phi, loc_x, loc_y + 1, h)/A
                       + (center_diff_x)*(center_diff_y) * grad_phi(phi, loc_x - 1, loc_y + 1, h)/A)
            
        #Upper left    
        elif cx < cell_x and cy > cell_y:
            force[i] += ((cell_x - center_diff_x )*(center_diff_y) * grad_phi(phi, loc_x + 1, loc_y, h)/A
                       + (center_diff_x)*(cell_y - center_diff_y) * grad_phi(phi, loc_x, loc_y - 1, h)/A
                       + (center_diff_x)*(center_diff_y) * grad_phi(phi, loc_x + 1, loc_y - 1, h)/A)
        
        #Lower left
        else:
            force[i] += ((cell_x - center_diff_x )*(center_diff_y) * grad_phi(phi, loc_x - 1, loc_y, h)/A
                       + (center_diff_x)*(cell_y - center_diff_y) * grad_phi(phi, loc_x, loc_y - 1, h)/A
                       + (center_diff_x)*(center_diff_y) * grad_phi(phi, loc_x - 1, loc_y - 1, h)/A)
    
    return force
    
forces = particle_forces(phi, points) 

#### Calculate New Positions and Set Up Plots

In [ ]:
def get_max_acc(forces):
    max_acc = 0
    for i in range(len(forces)):
        new_max = math.sqrt(forces[i][0]**2 + forces[i][1]**2)
        if new_max > max_acc:
            max_acc = new_max
    return max_acc

def time_step(max_acc):
    return .9*math.sqrt((1/1024)/(max_acc))

def new_positions(points, v0, forces, t_step):
    
    #Verlet method to update velocities and positions
    max_acc = get_max_acc(forces)
    t_step = time_step(max_acc)
    vh = np.zeros((Np, 2), dtype=float)
    vh = v0 + t_step/2 * forces
    points = (points + t_step*vh) % L #Deal with out of bounds
    v0 = vh + t_step/2 * forces
    
    return points, v0

def get_points(Np):
    points = []
    rand.seed(728)
    
    for i in range(Np):
        r = R*math.sqrt(rand.random())
        angle = 2*math.pi*rand.random()
        x = start + r*math.cos(angle)
        y = start + r*math.sin(angle)
        points.append((x,y))
        
    return points

def plot_points(points, title):
    x_points = []
    y_points = []
    
    for i in range(len(points)):
        x_points.append(points[i][0])
        y_points.append(points[i][1])

    plt.figure(figsize=(5,5)) #make all figures larger
    plt.scatter(x_points, y_points, s=1)
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.title("Simulation at " + title +" t_dyn")
    plt.axes().set_aspect('equal')
    plt.show()
    
def get_r(x, y):
    return math.sqrt((x-.5)**2 + (y-.5)**2)

def plot_avg_density(points, title):
    m = .1
    ring_width = L/1024
    rho_avg = np.zeros(1024)
    rho_r = np.zeros(1024)
    ring_area = 0
    
    for i in range(Np):
        
        r = get_r(points[i][0], points[i][1])
        ring_loc = abs(int(r // ring_width) - 512)
        
        if ring_loc > 1024:
            break
        if ring_loc == 0:
            ring_area = abs(math.pi * ring_width**2)
        else:
            ring_area = abs(math.pi * ring_width**2* (2*ring_loc - 1))
        rho_avg[ring_loc] += m / ring_area
        rho_r[ring_loc] = ring_loc * L/1024
      
    plt.plot(rho_r, rho_avg)
    plt.xlabel('r (m)')
    plt.ylabel(r'$\rho_{avg}\hspace{1}(\frac{kg}{m^2})$')
    plt.title("Average Density at " + title +" t_dyn")
    plt.show()
    
def get_acc(force_x, force_y):
    return math.sqrt(force_x**2 + force_y**2)
    
def plot_avg_acc(points, forces, title):
    m = .1
    ring_width = L/1024
    acc_avg = np.zeros(1024)
    acc_r = np.zeros(1024)
    ring_area = 0
    
    for i in range(Np):
        
        r = get_r(points[i][0], points[i][1])
        acc = get_acc(forces[i][0], forces[i][1])
        
        ring_loc = abs(int(r // ring_width) - 512)
        
        if ring_loc > 1024:
            break
        if ring_loc == 0:
            ring_area = abs(math.pi * ring_width**2)
        else:
            ring_area = abs(math.pi * ring_width**2* (2*ring_loc - 1))
        acc_avg[ring_loc] += acc / ring_area
        acc_r[ring_loc] = ring_loc * L/1024
      
    plt.plot(acc_r, acc_avg)
    plt.xlabel('r (m)')
    plt.ylabel(r'$a_{avg}\hspace{1}(\frac{m}{s^2})$')
    plt.title("Average Acceleration at " + title +" t_dyn")
    plt.show()

#### Run the simulation and display results

In [ ]:
def gravitational_collapse():
    
    #Set initial conditions
    Np = 4096
    v0 = np.zeros((Np, 2), dtype = float)
    points = get_points(Np)
    t = 0
   
    plot_points(points, title='0') #Initial points
    #plot_avg_density(points, title='0')  
    
    while t/t_dyn < 1.5:
        
        #Run all the calculations
        epsilon = 10**-2
        rho = density_field(points)
        rho_k = fft.rfft2(rho)
        phi_k = .1*rho_k * kernel * (L/N)**2 
        phi = fft.irfft2(phi_k)
        forces = particle_forces(phi, points)
       # if t == 0:
            #plot_avg_acc(points, forces, '0') 
        max_acc = get_max_acc(forces)
        t_step = time_step(max_acc) * 1/3
        points, v0 = new_positions(points, v0, forces, t_step)
        t += t_step
        
        #Plot points, avg density, and avg acceleration at certain intervals
        if abs(t/t_dyn - .5) < epsilon:
            plot_points(points, title='.5')
            #plot_avg_density(points, title='.5')
            #plot_avg_acc(points, forces, '.5')  
            
        if abs(t/t_dyn - 1) < epsilon and t/t_dyn - 1 < 0:
            plot_points(points, title='1')
            #plot_avg_density(points, title='1')
            #plot_avg_acc(points, forces, '1')  
               
    plot_points(points, title='1.5')
    #plot_avg_density(points, title='1.5')
    #plot_avg_acc(points, forces, '1.5')  

gravitational_collapse()

#### Discussion of Results

In a gravitational collapse, the particles are drawn towards their center of mass, as can be seen by the evolution of the particles over 1 t_dyn.  The particles continue to be drawn towards each other until all the mass is condensed over a small area, as can be seen at 1 t_dyn.  The particles then explode outwards, which occurs due to the extremely high density of particles gathered at the center. In terms of average density, the density is always larger closer to the center until the particles explode outwards, since there are a larger number of particles over a smaller area.  The average density closer to the center continues to increase, as can be seen.  When the particles explode outward, the density is far more uniform due to the particles being spread out instead of condensed at the center.  Acceleration is constantly constantly increasing as the particles are being drawn faster and faster towards the center until the particles explode outward, where the acceleration rapidly decreases as the particles get further and further away from each other. In the PM method, the resolution is limited by the cell length, so it can't represent the forces between particles accurately as they get close together.  A more accurate method would be able to model this and have correct interactions.